<h1>Contenido<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-Preprocesamiento" data-toc-modified-id="1.-Preprocesamiento-0">1. Preprocesamiento</a></span></li><li><span><a href="#2.-Procesamiento" data-toc-modified-id="2.-Procesamiento-1">2. Procesamiento</a></span></li></ul></div>

## 1. Preprocesamiento

In [8]:
%matplotlib inline
from pandas_profiling import ProfileReport
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 2. Procesamiento

In [13]:
meses = ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
filesUrlBase = 'https://raw.githubusercontent.com/jcmontoyar/CUSO2020/master/Data/'

Método genérico para leer cualquier archivo del censo, recibe el mes, el area y la categoria.
Retorna un dataframe del archivo escogido

In [31]:
def readMonthAreaCategory(mes, area, category):
    fileUrl = filesUrlBase + mes + '.csv' + '/' + area + ' - ' + category + '.csv'
    fileUrl = fileUrl.replace(" ", "%20")
    return pd.read_csv(fileUrl,delimiter=';', encoding='utf-8')

Método para leer ocupados de cabecera

In [107]:
estratos = ['Sin conexion, pirata y estrato 1', 'Estrato 2', ' Estrato 3', 'Estrato 4, 5 y 6','Sin informacion']
# Recibe un dataframe de categoria ocupados y le añade una columna indicando el estrato
def setEstrato(df):
    conditions = [
        (df['P4030S1'] == 1) & (df['P4030S1A1'] == 0),
        (df['P4030S1'] == 1) & (df['P4030S1A1'] == 1),
        (df['P4030S1'] == 2),
        (df['P4030S1'] == 1) & (df['P4030S1A1'] == 2),
        (df['P4030S1'] == 1) & (df['P4030S1A1'] == 3),
        (df['P4030S1'] == 1) & (df['P4030S1A1'] == 4),
        (df['P4030S1'] == 1) & (df['P4030S1A1'] == 5),
        (df['P4030S1'] == 1) & (df['P4030S1A1'] == 6),
        (df['P4030S1'] == 1) & (df['P4030S1A1'] == 9)]
    choices = [estratos[0], estratos[0], estratos[0], estratos[1],estratos[2],estratos[3],estratos[3],estratos[3], estratos[4]]
    df['Estrato'] = np.select(conditions, choices, default = estratos[4])
    return df
         
def readOcupadosCabecera(mes):
    #Obtenener data frame
    df = readMonthAreaCategory(mes, 'Cabecera', 'Ocupados')
    return df

# Read vivienda y hogares given the month.
def readViviendaYHogaresCabecera(mes):
    #Obtenener data frame
    df = readMonthAreaCategory(mes, 'Cabecera', 'Vivienda y Hogares')
    
    #Fix null value in P4030S1A1 with 9 (no information)
    df['P4030S1A1'] = df['P4030S1A1'].replace(to_replace =" ",  value ="9").astype(int)
    
    #Set estratos
    df = setEstrato(df)
    return df

In [108]:
df = readViviendaYHogaresCabecera('Julio')

In [109]:
ProfileReport(df)

C:\Users\julia\Anaconda3\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,61
Number of observations,17743
Total Missing (%),0.0%
Total size in memory,8.2 MiB
Average record size in memory,484.0 B
Numeric,15
Categorical,14
Boolean,28
Date,0
Text (Unique),0
Rejected,4


In [110]:
dfs = []
for i in meses:
    dfs.append(readViviendaYHogaresCabecera(i))

C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (5,47) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (47,53) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [114]:
dfOcupados = []
for i in meses:
    dfOcupados.append(readOcupadosCabecera(i))

C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (96,165) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (165) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (96) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [116]:
dfCaracteristicasGen = []
for i in meses:
    dfOcupados.append(readMonthAreaCategory(i,'Cabecera','Caracteristicas generales (Personas)'))

C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


HTTPError: HTTP Error 404: Not Found